In [26]:
!pip install nltk

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
import re

In [2]:
df=pd.read_csv("다시 크롤링/N Seoul Tower.csv")
df["score"]=(df['STAR']>3.0)
df=df.replace(True,1)
df=df.replace(False,0)
print(df['score'].sum())
df

4515


,Unnamed: 0,STAR,CONTENT,score
0,0,5.0,Seoul tower is very peacfull and one of clean ...,1
1,1,4.0,We have walked up and down It is a good walk ...,1
2,2,4.0,It was a bit late to open and it was raining ...,1
3,3,5.0,It was nice to be able to see the beautiful n...,1
4,4,5.0,Got there at dusk By the time we got into the ...,1
...,...,...,...,...
5503,5503,3.0,when I visit here there is no cherry blossoms ...,0
5504,5504,4.0,It gives a beautiful view of the city. It is b...,1
5505,5505,4.0,"great view from the observatory, yet Tokyo tow...",1
5506,5506,5.0,There are good trails around here and amazing ...,1


In [3]:
def text_clearing(text):
    eng=re.compile('[^ a-z A-Z 0-9]+')
    result=eng.sub('',text)
    return result

In [4]:
df["CONTENT"]=df["CONTENT"].apply(lambda x:text_clearing(x))
df

,Unnamed: 0,STAR,CONTENT,score
0,0,5.0,Seoul tower is very peacfull and one of clean ...,1
1,1,4.0,We have walked up and down It is a good walk ...,1
2,2,4.0,It was a bit late to open and it was raining ...,1
3,3,5.0,It was nice to be able to see the beautiful n...,1
4,4,5.0,Got there at dusk By the time we got into the ...,1
...,...,...,...,...
5503,5503,3.0,when I visit here there is no cherry blossoms ...,0
5504,5504,4.0,It gives a beautiful view of the city It is be...,1
5505,5505,4.0,great view from the observatory yet Tokyo towe...,1
5506,5506,5.0,There are good trails around here and amazing ...,1


In [5]:
import nltk
stopwords=nltk.corpus.stopwords.words('english')
def get_pos (x) :
    token=nltk.word_tokenize(x)
    pos = nltk.pos_tag(token)
    result = []
    
    for a1 in pos :
        if a1[0] not in stopwords:
            result.append(f'{a1[0]}/{a1[1]}')
    
    return result

In [6]:
index_vectorizer=CountVectorizer(tokenizer=lambda x:get_pos(x))
X=index_vectorizer.fit_transform(df["CONTENT"].tolist())
X

<5508x14896 sparse matrix of type '<class 'numpy.int64'>'
	with 171525 stored elements in Compressed Sparse Row format>

In [7]:
index_vectorizer.vocabulary_

{'seoul/NN': 11282,
 'tower/NN': 13348,
 'peacfull/JJ': 9473,
 'one/CD': 8979,
 'clean/JJ': 2669,
 'place/NN': 9714,
 'visit/NN': 14089,
 'life/NN': 7314,
 'beautiful/JJ': 1594,
 'city/NN': 2635,
 'view/NN': 14027,
 'fantastic/JJ': 4813,
 'gardens/NNS': 5363,
 'trees/NNS': 13518,
 'highly/RB': 5994,
 'suggest/VB': 12499,
 'visit/VB': 14094,
 'visit/VBP': 14097,
 'seoul/JJ': 11281,
 'must/MD': 8313,
 'add/VB': 710,
 'booking/VBG': 1867,
 'list/NN': 7404,
 'come/VBP': 2878,
 'love/VB': 7599,
 'spend/VB': 12042,
 'time/NN': 13141,
 'walked/VBN': 14221,
 'good/JJ': 5535,
 'walk/NN': 14205,
 'side/NN': 11551,
 'stairs/NN': 12175,
 'path/NN': 9430,
 'forest/JJS': 5153,
 'visibility/NN': 14080,
 'bit/NN': 1762,
 'poor/JJ': 9869,
 'expected/VBN': 4659,
 'cloudy/NN': 2772,
 'weather/NN': 14366,
 'late/JJ': 7168,
 'open/VB': 9025,
 'raining/VBG': 10354,
 'hard/JJ': 5811,
 'wait/VB': 14187,
 'outside/JJ': 9161,
 'still/RB': 12313,
 'nice/JJ': 8600,
 'able/JJ': 619,
 'see/VB': 11207,
 'night/NN': 

In [8]:
y=df["score"]

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(X)
for i in range(10):
    print(X[i])
    print("*"*20)


  (0, 14097)	0.2098262825139298
  (0, 14094)	0.122430757734026
  (0, 14089)	0.13822718886446572
  (0, 14027)	0.06796413159126279
  (0, 13518)	0.1844416672326723
  (0, 13348)	0.06031606854790199
  (0, 13141)	0.10442581099573818
  (0, 12499)	0.23416067562244589
  (0, 12042)	0.18722107425393605
  (0, 11282)	0.07250687929147867
  (0, 11281)	0.1468806238035829
  (0, 9714)	0.19313266525934916
  (0, 9473)	0.3569054088742604
  (0, 8979)	0.11680457492410212
  (0, 8313)	0.12986197565077212
  (0, 7599)	0.1849823156143754
  (0, 7404)	0.2459814221232007
  (0, 7314)	0.24017307153470308
  (0, 5994)	0.19445775730138778
  (0, 5363)	0.26283306565933534
  (0, 4813)	0.183644151693561
  (0, 2878)	0.20505786323704756
  (0, 2669)	0.21244207484693817
  (0, 2635)	0.08763894569665466
  (0, 1867)	0.31295281901653044
  (0, 1594)	0.12152914501003481
  (0, 710)	0.23705403519018078
********************
  (0, 14366)	0.20472225944031772
  (0, 14221)	0.3169753871833222
  (0, 14205)	0.1661316962540277
  (0, 14080)	0.290

In [10]:
print(X[0])
print(X.shape[0])

  (0, 14097)	0.2098262825139298
  (0, 14094)	0.122430757734026
  (0, 14089)	0.13822718886446572
  (0, 14027)	0.06796413159126279
  (0, 13518)	0.1844416672326723
  (0, 13348)	0.06031606854790199
  (0, 13141)	0.10442581099573818
  (0, 12499)	0.23416067562244589
  (0, 12042)	0.18722107425393605
  (0, 11282)	0.07250687929147867
  (0, 11281)	0.1468806238035829
  (0, 9714)	0.19313266525934916
  (0, 9473)	0.3569054088742604
  (0, 8979)	0.11680457492410212
  (0, 8313)	0.12986197565077212
  (0, 7599)	0.1849823156143754
  (0, 7404)	0.2459814221232007
  (0, 7314)	0.24017307153470308
  (0, 5994)	0.19445775730138778
  (0, 5363)	0.26283306565933534
  (0, 4813)	0.183644151693561
  (0, 2878)	0.20505786323704756
  (0, 2669)	0.21244207484693817
  (0, 2635)	0.08763894569665466
  (0, 1867)	0.31295281901653044
  (0, 1594)	0.12152914501003481
  (0, 710)	0.23705403519018078
5508


In [11]:
params = {'penalty' : ['l1', 'l2', 'elasticnet', 'none'], 'C' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000] }

model2 = LogisticRegression()
kfold = KFold(n_splits=10, shuffle=True, random_state = 1)
grid_clf2 = GridSearchCV(model2, param_grid=params, scoring='f1', cv=kfold)
grid_clf2.fit(X, y)

print(f'최적의 하이퍼 파라미터 : {grid_clf2.best_params_}')
print(f'최적의 모델 평균 성능 : {grid_clf2.best_score_}')

최적의 하이퍼 파라미터 : {'C': 10, 'penalty': 'l2'}
최적의 모델 평균 성능 : 0.9124608519769577


In [12]:
model = grid_clf2.best_estimator_

a1 = (model.coef_[0])
display(a1)
len(a1)

array([ 0.19949828,  0.23491836,  0.93513836, ..., -0.74350812,
        0.09662861,  0.09662861])

14896

In [13]:
a2 = list(enumerate(a1))
a2

[(0, 0.19949828001496417),
 (1, 0.23491835563427937),
 (2, 0.9351383569269096),
 (3, 0.8060061734489756),
 (4, 0.8060061734489756),
 (5, -1.107280000490116),
 (6, 0.8503786527930902),
 (7, 0.10435227579838681),
 (8, 0.5485643029120651),
 (9, 0.15766999498380907),
 (10, 0.04543583981788639),
 (11, -2.0392588309056614),
 (12, 0.028103362425999662),
 (13, 0.005828332455484789),
 (14, -0.1854389237560346),
 (15, -0.4357964322616286),
 (16, 0.34200130901782827),
 (17, -0.3594970866673081),
 (18, 0.10435227579838681),
 (19, 0.022281406494639045),
 (20, -0.17540716698867675),
 (21, 0.26589714118661634),
 (22, -0.29520069343948685),
 (23, 0.16223883633350225),
 (24, -1.9839842310961724),
 (25, -0.26988096473923473),
 (26, 0.05204778991888148),
 (27, 0.2530153320965016),
 (28, 0.09058004072910704),
 (29, 0.019365388374588316),
 (30, -0.4112025190961158),
 (31, 0.176607569200851),
 (32, -1.0148070044462125),
 (33, 0.08446349825868),
 (34, 0.053753226139431634),
 (35, 0.041716962712017816),
 (36,

In [14]:
a3 = []

for idx, value in a2 : 
    a3.append((value, idx))
    
a3

[(0.19949828001496417, 0),
 (0.23491835563427937, 1),
 (0.9351383569269096, 2),
 (0.8060061734489756, 3),
 (0.8060061734489756, 4),
 (-1.107280000490116, 5),
 (0.8503786527930902, 6),
 (0.10435227579838681, 7),
 (0.5485643029120651, 8),
 (0.15766999498380907, 9),
 (0.04543583981788639, 10),
 (-2.0392588309056614, 11),
 (0.028103362425999662, 12),
 (0.005828332455484789, 13),
 (-0.1854389237560346, 14),
 (-0.4357964322616286, 15),
 (0.34200130901782827, 16),
 (-0.3594970866673081, 17),
 (0.10435227579838681, 18),
 (0.022281406494639045, 19),
 (-0.17540716698867675, 20),
 (0.26589714118661634, 21),
 (-0.29520069343948685, 22),
 (0.16223883633350225, 23),
 (-1.9839842310961724, 24),
 (-0.26988096473923473, 25),
 (0.05204778991888148, 26),
 (0.2530153320965016, 27),
 (0.09058004072910704, 28),
 (0.019365388374588316, 29),
 (-0.4112025190961158, 30),
 (0.176607569200851, 31),
 (-1.0148070044462125, 32),
 (0.08446349825868, 33),
 (0.053753226139431634, 34),
 (0.041716962712017816, 35),
 (-2.

In [15]:
coef_pos_index = sorted(a3, reverse=True)
coef_pos_index

[(6.131436138274444, 14618),
 (5.6570884503923, 972),
 (5.108928078064712, 974),
 (4.685899584266481, 4572),
 (4.612886267748095, 1594),
 (4.50184084292775, 5606),
 (4.2936976117409245, 7607),
 (4.216394706853723, 11282),
 (4.159639343871775, 4813),
 (4.063565920067563, 2635),
 (4.034196074226441, 7290),
 (3.502210517711825, 2574),
 (3.454318490204182, 8313),
 (3.3495334570802533, 820),
 (3.3360572061000475, 7712),
 (3.272858093389362, 12022),
 (3.2608649928645135, 13510),
 (3.2009556024576833, 12458),
 (3.158038494148518, 4373),
 (3.114423693807384, 12343),
 (3.108779109543745, 2801),
 (3.068888302198722, 10445),
 (3.055857666484303, 7602),
 (3.0109534725272664, 4374),
 (2.957782607214886, 13829),
 (2.957243104643917, 10239),
 (2.921304875427135, 5303),
 (2.8677277230759826, 5809),
 (2.84583600392561, 7169),
 (2.841491000129768, 4235),
 (2.7573265160158162, 7314),
 (2.7442220458608704, 4055),
 (2.68586716390162, 13048),
 (2.6731281543098233, 3643),
 (2.6690908998325362, 14691),
 (2.66

In [16]:
text_data_dict = {}

for key in index_vectorizer.vocabulary_ :
    value = index_vectorizer.vocabulary_[key]
    text_data_dict[value] = key
    
text_data_dict

{11282: 'seoul/NN',
 13348: 'tower/NN',
 9473: 'peacfull/JJ',
 8979: 'one/CD',
 2669: 'clean/JJ',
 9714: 'place/NN',
 14089: 'visit/NN',
 7314: 'life/NN',
 1594: 'beautiful/JJ',
 2635: 'city/NN',
 14027: 'view/NN',
 4813: 'fantastic/JJ',
 5363: 'gardens/NNS',
 13518: 'trees/NNS',
 5994: 'highly/RB',
 12499: 'suggest/VB',
 14094: 'visit/VB',
 14097: 'visit/VBP',
 11281: 'seoul/JJ',
 8313: 'must/MD',
 710: 'add/VB',
 1867: 'booking/VBG',
 7404: 'list/NN',
 2878: 'come/VBP',
 7599: 'love/VB',
 12042: 'spend/VB',
 13141: 'time/NN',
 14221: 'walked/VBN',
 5535: 'good/JJ',
 14205: 'walk/NN',
 11551: 'side/NN',
 12175: 'stairs/NN',
 9430: 'path/NN',
 5153: 'forest/JJS',
 14080: 'visibility/NN',
 1762: 'bit/NN',
 9869: 'poor/JJ',
 4659: 'expected/VBN',
 2772: 'cloudy/NN',
 14366: 'weather/NN',
 7168: 'late/JJ',
 9025: 'open/VB',
 10354: 'raining/VBG',
 5811: 'hard/JJ',
 14187: 'wait/VB',
 9161: 'outside/JJ',
 12313: 'still/RB',
 8600: 'nice/JJ',
 619: 'able/JJ',
 11207: 'see/VB',
 8629: 'night

In [17]:
top50 = coef_pos_index[:50]
bottom50 = coef_pos_index[-50:]

In [18]:
import nltk
from nltk.corpus import stopwords

In [19]:
nltk.download('english')
stop_words = set(stopwords.words('english'))
add_stop_words = ["'", ",", "'s", ":", ".", "?", "!","(",")","n't",
                 "korea","korean","good","like","awesome","perfect","nice","well","expensive","great","seoul","beautiful",
                 'wonderful','view','views','city','place','visit','tower','just',
                 'lotte','world','ride','rides',
                  'went','lotte','world','tower',
                  'street','lot','lots',
                  'market','ddp','insadong','places','shops','bookstore','bookstores','book','books','kyobo',
                  'lotte','aquarium','feel','animals','park','animals','coex','mall','shopping',
                  'amazing','6','1','40','8pm','1','40'
                 ]
for word in add_stop_words:
    stop_words.add(word)

[nltk_data] Error loading english: Package 'english' not found in
[nltk_data]     index


In [20]:
pos=[]
for value, idx in top50 :
    if text_data_dict[idx].split('/')[0] not in stop_words:
        pos.append(text_data_dict[idx])
        print(text_data_dict[idx])

excellent/JJ
loved/VBD
fantastic/JJ
level/NN
choose/VB
must/MD
afternoon/NN
magnificent/JJ
spectacular/JJ
tree/NN
stunning/VBG
enjoyed/VBD
stops/NNS
coffee/NN
reasonably/RB
love/VBP
enjoyed/VBN
uphill/NN
put/VB
fun/NN
happy/JJ
late/RB
either/DT
life/NN
drink/NN
thousands/NNS
depending/VBG
worth/NN
stunning/JJ
buffet/NN
anywhere/RB
choice/NN
shop/NN
either/RB
times/NNS
less/RBR
buy/VB
360/CD
yellow/JJ
love/IN


In [21]:
neg=[]
for value, idx in bottom50 :
    if text_data_dict[idx].split('/')[0] not in stop_words:
        neg.append(text_data_dict[idx])
        print(text_data_dict[idx])

guess/NN
fine/JJ
fingerprints/NNS
queuing/VBG
interest/NN
dont/JJ
imagine/VB
touristic/JJ
filled/VBN
boring/NN
minutes/NNS
typical/JJ
expectation/NN
solo/VBZ
smog/JJ
pricy/NN
driving/VBG
standing/VBG
overcast/JJ
unfortunately/RB
wasnt/VBP
ok/JJ
lock/VB
slope/NN
limited/JJ
enough/RB
reduces/VBZ
give/VBP
landmark/NN
group/NN
expected/VBD
alone/NN
dissepoinding/VBG
else/RB
much/RB
crowded/VBN
crazy/JJ
better/JJR
overview/VB
closed/VBN
smog/NN
unimpressive/JJ
tourists/NNS
average/JJ
overview/NN
much/JJ
crowded/JJ
disappointed/JJ
nothing/NN


In [22]:
num_pos_rev=[]        
for i in pos:
    num_pos_rev.append(sum(df['CONTENT'].str.contains(i.split('/')[0])))

In [23]:
num_neg_rev=[]        
for i in neg:
    num_neg_rev.append(sum(df['CONTENT'].str.contains(i.split('/')[0])))

In [24]:
revword=pd.DataFrame([pos,num_pos_rev,neg,num_neg_rev],index=['긍정','리뷰 수','부정', '리뷰 수']).transpose()
revword

,긍정,리뷰 수,부정,리뷰 수
0,excellent/JJ,76,guess/NN,58
1,loved/VBD,144,fine/JJ,54
2,fantastic/JJ,141,fingerprints/NNS,8
3,level/NN,169,queuing/VBG,29
4,choose/VB,94,interest/NN,201
5,must/MD,553,dont/JJ,410
6,afternoon/NN,162,imagine/VB,42
7,magnificent/JJ,48,touristic/JJ,11
8,spectacular/JJ,209,filled/VBN,37
9,tree/NN,262,boring/NN,22


In [25]:
revword.to_csv("N Seoul Tower words.csv")

In [26]:
keyword=input("키워드 입력 : ")
df_a = df[df['CONTENT'].str.contains(keyword)].sort_values(by=['STAR'])
df_a.to_csv(keyword +"키워드 포함 리뷰")
df_a

키워드 입력 : smog


,Unnamed: 0,STAR,CONTENT,score
2645,2645,2.0,If you are feeling energetic you can walk up t...,0
4358,4358,2.0,If you are first time in Seoul you should go u...,0
5157,5157,2.0,I would not really recommend here unless you h...,0
2532,2532,2.0,This is our second time there in ten years Nam...,0
2677,2677,2.0,This was a disappointment There is nothing esp...,0
...,...,...,...,...
4652,4652,5.0,This is one of the must see attractions in Seo...,1
2566,2566,5.0,This was my second attempt at checking out the...,1
2354,2354,5.0,Something really cool about looking down on a ...,1
932,932,5.0,I say great view of Seoul IF the weather or sm...,1
